In [1]:
from PIL import Image
import pytesseract

In [12]:
im = Image.open('yanzenma.jpg')

In [3]:
im.size

(160, 70)

In [13]:
t = pytesseract.image_to_string(im)

In [7]:
im.show()

In [16]:
t

u'38-3'